In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
IMG_DIR = 'camera_cal'
NX, NY = 9, 6  # number of points on a chessboard

objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane.

objp = np.zeros((NX * NY, 3), np.float32)
objp[:, :2] = np.mgrid[0:NX, 0:NY].T.reshape(-1, 2)

# get the list of calibration images
images = [os.path.join(IMG_DIR, filename) for filename in os.listdir(IMG_DIR)]

for image in images:
    img = cv2.imread(image)

    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (NX, NY), None)
    
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

In [ ]:
test_image = 'camera_cal/calibration1.jpg'
img = cv2.imread(test_image)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1], img.shape[0]), None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=20)

In [ ]:
test_image = 'test_images/test1.jpg'
img = cv2.imread(test_image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (img.shape[1], img.shape[0]), None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=20)

In [ ]:
test_image = 'test_images/test1.jpg'
img = cv2.imread(test_image)
hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(30, 10))

for i in range(3):
    ax = plt.subplot(2, 3, i + 1)
    ax.set_title('RGB'[i], fontsize=20)
    ax.imshow(rgb[:, :, i], cmap='gray')

for i in range(3):
    ax = plt.subplot(2, 3, i + 4)
    ax.set_title('HLS'[i], fontsize=20)
    ax.imshow(hls[:, :, i], cmap='gray')

In [ ]:
s = hls[:, :, 2]
thresh = (90, 255)
s_binary = np.zeros_like(s)
s_binary[(s >= thresh[0]) & (s <= thresh[1])] = 1
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(s, cmap='gray')
ax1.set_title('S', fontsize=20)
ax2.imshow(s_binary, cmap='gray')
ax2.set_title('binary S, threshold = {}:{}'.format(*thresh), fontsize=20)

In [ ]:
r = rgb[:, :, 0]
thresh = (220, 255)
binary = np.zeros_like(r)
binary[(r >= thresh[0]) & (r <= thresh[1])] = 1
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(r, cmap='gray')
ax1.set_title('R', fontsize=20)
ax2.imshow(binary, cmap='gray')
ax2.set_title('binary R, threshold = {}:{}'.format(*thresh), fontsize=20)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sobel = cv2.Sobel(s, cv2.CV_64F, 1, 0)
sobel = np.absolute(sobel)
sobel = np.uint8(255 * sobel / np.max(sobel))
thresh = (30, 100)
sobel_binary = np.zeros_like(sobel)
sobel_binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(sobel, cmap='gray')
ax1.set_title('gradient', fontsize=20)
ax2.imshow(sobel_binary, cmap='gray')
ax2.set_title('binary gradient, threshold = {}:{}'.format(*thresh), fontsize=20)

In [ ]:
color_binary = np.dstack((255 * s_binary, 255 * sobel_binary, np.zeros_like(s_binary)))

# Combine the two binary thresholds
combined_binary = np.zeros_like(s_binary)
combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1

# Plotting thresholded images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.set_title('Stacked thresholds', fontsize=20)
ax1.imshow(color_binary, cmap='gray')

ax2.set_title('Combined S channel and gradient thresholds', fontsize=20)
ax2.imshow(combined_binary, cmap='gray')

In [ ]:
def pipeline(image, mtx, dist):
    img = np.copy(image)
    img = cv2.undistort(img, mtx, dist, None, mtx)

    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s = hls[:, :, 2]
    thresh = (90, 255)
    s_binary = np.zeros_like(s)
    s_binary[(s >= thresh[0]) & (s <= thresh[1])] = 1

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(s, cv2.CV_64F, 1, 0)
    sobel = np.absolute(sobel)
    sobel = np.uint8(255 * sobel / np.max(sobel))
    thresh = (20, 100)
    sobel_binary = np.zeros_like(sobel)
    sobel_binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1

    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1
    return combined_binary

In [ ]:
test_image = 'test_images/straight_lines1.jpg'
img = cv2.imread(test_image)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.set_title('original image', fontsize=20)
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

ax2.set_title('undistorted, combined s channel and gradient thresholds', fontsize=20)
ax2.imshow(pipeline(img, mtx, dist), cmap='gray')

In [ ]:
from matplotlib.path import Path
import matplotlib.patches as patches
height = img.shape[0]

src = np.float32([(190, 719), (577, 460), (705, 460), (1120, 719)])
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.plot([src[i][0] for i in range(4)],
         [src[i][1] for i in range(4)],
         'r-', lw=2)

In [ ]:
#src = np.float32([(190, 719), (534, 490), (753, 490), (1120, 719)])
dst = np.float32([(190, 719), (190, 0), (1120, 0), (1120, 719)])
M = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
plt.figure(figsize=(20, 10))
plt.imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
plt.plot([dst[i][0] for i in range(4)], [dst[i][1] for i in range(4)], 'r-', lw=2)

In [ ]:
def pipeline(image, mtx, dist, M):
    img = np.copy(image)
    img = cv2.undistort(img, mtx, dist, None, mtx)

    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s = hls[:, :, 2]
    thresh = (90, 255)
    s_binary = np.zeros_like(s)
    s_binary[(s >= thresh[0]) & (s <= thresh[1])] = 1

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobel = cv2.Sobel(s, cv2.CV_64F, 1, 0)
    sobel = np.absolute(sobel)
    sobel = np.uint8(255 * sobel / np.max(sobel))
    thresh = (30, 100)
    sobel_binary = np.zeros_like(sobel)
    sobel_binary[(sobel >= thresh[0]) & (sobel <= thresh[1])] = 1

    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 1) | (sobel_binary == 1)] = 1
    
    warped = cv2.warpPerspective(combined_binary, M, (img.shape[1], img.shape[0]))
    return warped

In [ ]:
test_image = 'test_images/straight_lines1.jpg'
img = cv2.imread(test_image)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.set_title('original image', fontsize=20)
ax1.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

ax2.set_title('undistorted, combined s channel and gradient thresholds', fontsize=20)
ax2.imshow(pipeline(img, mtx, dist, M), cmap='gray')